## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
%matplotlib inline
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Nioro,GM,13.3500,-15.7500,75.52,47,99,3.87,overcast clouds
1,1,Kapaa,US,22.0752,-159.3190,82.38,75,90,1.99,overcast clouds
2,2,Puerto Ayora,EC,-0.7393,-90.3518,68.38,75,35,8.34,scattered clouds
3,3,Cape Town,ZA,-33.9258,18.4232,63.10,65,0,20.71,clear sky
4,4,Rikitea,PF,-23.1203,-134.9692,75.81,72,0,10.65,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        161
City           161
Country        161
Lat            161
Lng            161
Max Temp       161
Humidity       161
Cloudiness     161
Wind Speed     161
Description    161
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Nioro,GM,75.52,overcast clouds,13.3500,-15.7500,
1,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,
4,Rikitea,PF,75.81,clear sky,-23.1203,-134.9692,
5,Labuhan,ID,83.17,light rain,-6.8844,112.2051,
10,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,
13,Hilo,US,84.27,moderate rain,19.7297,-155.0900,
14,Butaritari,KI,81.95,light rain,3.0707,172.7902,
25,Katobu,ID,89.13,clear sky,-4.9408,122.5276,
27,Kon Tum,VN,75.24,broken clouds,14.3500,108.0000,
30,Salalah,OM,82.49,clear sky,17.0151,54.0924,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found ... skipping")

Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping
Hotel not found ... skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Nioro,GM,75.52,overcast clouds,13.3500,-15.7500,
1,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Rikitea,PF,75.81,clear sky,-23.1203,-134.9692,People ThankYou
5,Labuhan,ID,83.17,light rain,-6.8844,112.2051,PT PPI Lamongan Tambak C
10,Carnarvon,AU,78.87,clear sky,-24.8667,113.6333,Hospitality Carnarvon
...,...,...,...,...,...,...,...
660,Dharmabad,IN,76.57,overcast clouds,18.9000,77.8500,Ismail khan
663,Galesong,ID,89.62,clear sky,-5.3166,119.3661,Sapriadi Appy
666,Bairiki,KI,82.40,broken clouds,1.3292,172.9752,Tarawa Boutique Hotel
669,Ruteng,ID,77.20,few clouds,-8.6136,120.4721,Hotel FX 72


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))